 # Table of Contents
<div class="toc" style="margin-top: 1em;"><ul class="toc-item" id="toc-level0"><ul class="toc-item"><li><span><a href="http://localhost:8888/notebooks/notebooks/io.ipynb#Import" data-toc-modified-id="Import-0.1"><span class="toc-item-num">0.1&nbsp;&nbsp;</span>Import</a></span><ul class="toc-item"><li><span><a href="http://localhost:8888/notebooks/notebooks/io.ipynb#Common" data-toc-modified-id="Common-0.1.1"><span class="toc-item-num">0.1.1&nbsp;&nbsp;</span>Common</a></span></li><li><span><a href="http://localhost:8888/notebooks/notebooks/io.ipynb#Libs" data-toc-modified-id="Libs-0.1.2"><span class="toc-item-num">0.1.2&nbsp;&nbsp;</span>Libs</a></span></li><li><span><a href="http://localhost:8888/notebooks/notebooks/io.ipynb#Special" data-toc-modified-id="Special-0.1.3"><span class="toc-item-num">0.1.3&nbsp;&nbsp;</span>Special</a></span></li></ul></li><li><span><a href="http://localhost:8888/notebooks/notebooks/io.ipynb#Load-longitudinal-retention" data-toc-modified-id="Load-longitudinal-retention-0.2"><span class="toc-item-num">0.2&nbsp;&nbsp;</span>Load longitudinal retention</a></span><ul class="toc-item"><li><span><a href="http://localhost:8888/notebooks/notebooks/io.ipynb#Move" data-toc-modified-id="Move-0.2.1"><span class="toc-item-num">0.2.1&nbsp;&nbsp;</span>Move</a></span></li><li><span><a href="http://localhost:8888/notebooks/notebooks/io.ipynb#Convert" data-toc-modified-id="Convert-0.2.2"><span class="toc-item-num">0.2.2&nbsp;&nbsp;</span>Convert</a></span></li></ul></li><li><span><a href="http://localhost:8888/notebooks/notebooks/io.ipynb#Load-longitudinal-seq=1" data-toc-modified-id="Load-longitudinal-seq=1-0.3"><span class="toc-item-num">0.3&nbsp;&nbsp;</span>Load longitudinal seq=1</a></span><ul class="toc-item"><li><span><a href="http://localhost:8888/notebooks/notebooks/io.ipynb#Convert-to-parq" data-toc-modified-id="Convert-to-parq-0.3.1"><span class="toc-item-num">0.3.1&nbsp;&nbsp;</span>Convert to parq</a></span></li></ul></li><li><span><a href="http://localhost:8888/notebooks/notebooks/io.ipynb#Load-longitudinal-seq=2" data-toc-modified-id="Load-longitudinal-seq=2-0.4"><span class="toc-item-num">0.4&nbsp;&nbsp;</span>Load longitudinal seq=2</a></span><ul class="toc-item"><li><span><a href="http://localhost:8888/notebooks/notebooks/io.ipynb#Move-raw" data-toc-modified-id="Move-raw-0.4.1"><span class="toc-item-num">0.4.1&nbsp;&nbsp;</span>Move raw</a></span></li><li><span><a href="http://localhost:8888/notebooks/notebooks/io.ipynb#Convert-to-parq" data-toc-modified-id="Convert-to-parq-0.4.2"><span class="toc-item-num">0.4.2&nbsp;&nbsp;</span>Convert to parq</a></span><ul class="toc-item"><li><span><a href="http://localhost:8888/notebooks/notebooks/io.ipynb#debug" data-toc-modified-id="debug-0.4.2.1"><span class="toc-item-num">0.4.2.1&nbsp;&nbsp;</span>debug</a></span></li><li><span><a href="http://localhost:8888/notebooks/notebooks/io.ipynb#debug-end" data-toc-modified-id="debug-end-0.4.2.2"><span class="toc-item-num">0.4.2.2&nbsp;&nbsp;</span>debug end</a></span></li></ul></li></ul></li><li><span><a href="http://localhost:8888/notebooks/notebooks/io.ipynb#Load-Onboarding-A/B" data-toc-modified-id="Load-Onboarding-A/B-0.5"><span class="toc-item-num">0.5&nbsp;&nbsp;</span>Load Onboarding A/B</a></span></li><li><span><a href="http://localhost:8888/notebooks/notebooks/io.ipynb#OOM" data-toc-modified-id="OOM-0.6"><span class="toc-item-num">0.6&nbsp;&nbsp;</span>OOM</a></span></li></ul></ul></div>

## Import

### Common

In [1]:
import boot_utes as bu; bu.reload(bu); from boot_utes import (reload, add_path, path)
add_path('../src/', '~/repos/myutils/')

11

In [2]:
from collections import defaultdict, Counter, OrderedDict
from functools import partial
from itertools import count
from operator import itemgetter as itg
from pprint import pprint
import os, sys, itertools as it, simplejson, time
from os.path import join
from glob import glob

### Libs

In [3]:
# from gensim.models import Word2Vec

import feather
import numpy as np
import numpy.random as nr
import pandas as pd
from pandas import DataFrame, Series
from pandas.compat import lmap, lrange, lfilter, lzip

import toolz.curried as z

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
pd.options.mode.use_inf_as_na = True

# %mkdir cache
import joblib; mem = joblib.Memory(location='cache')
dbx = '/Users/wbeard/miniconda3/envs/tmpdecomp2/bin/databricks'

In [4]:
def norm_rows(df):
    return df.div(df.sum(axis=1), axis=0)

In [28]:
re.compile(r'part.+?\.parquet')

re.compile(r'part.+?\.parquet', re.UNICODE)

In [30]:
re.compile(r'part.+?\.{}'.format('parquet'))

re.compile(r'part.+?\.parquet', re.UNICODE)

In [31]:
from os.path import join
import re

def get_spk_parq_fn(dir, ext='parquet'):
    s3parq_fs = !aws s3 ls $dir/
#     print(dir)
#     print(s3parq_fs)
    spk_parq_re = re.compile(r'part.+?\.{}'.format(ext))
    fn_all = spk_parq_re.findall(s3parq_fs[-1])
    if not len(fn_all) == 1:
        print('Uh oh: fn_all=', fn_all)
    [fn] = fn_all
    return fn

def show_s3_opts(*dir_elems):
    dir = join(*dir_elems)
    if not dir.endswith('/'):
        dir += '/'
    res = !aws s3 ls $dir
    return res
    
def mv_s3_pq(from_dir='', to_dir='data/pings', fn='pings_0125.parq', s3base=None, ext='parquet'):
    s3base = s3base or 's3://mozilla-databricks-telemetry-test/wbeard/'
    s3dir = join(s3base, from_dir, fn)
    
    try:
        pqn_ = get_spk_parq_fn(s3dir, ext=ext)
        print('Found pq file', pqn_)
    except IndexError:
        print('ERROR: File {} not found'.format(s3dir))
        opts = show_s3_opts(s3base, from_dir)
        print('\nPossible options:\n\t', '\n\t'.join(opts))
        return
        
    
    s3loc = join(s3base, from_dir, fn, pqn_)
    to_loc = join(to_dir, fn)
    
    
#     print(s3loc)
#     print(to_loc)
    !aws s3 cp $s3loc $to_loc
    return to_loc
    


### Special

In [5]:
# import utils.seq_utils as su; reload(su)
# import utils.events_utils as eu; reload(eu)
# import utils.feather_counter as fc; reload(fc)
# import utils.mutes as mt; reload(mt)

import myutils as mu
import functoolz as fz; reload(fz)
import numba_utils as nu; reload(nu)

mt.set_mem(mem=mem, json=simplejson)
su.set_memo(memoizer=mt.json_memo())
;;

NameError: name 'mt' is not defined

In [9]:
def move_new(dbx_dir, outdir=None, outdir_base='../data/raw/'):
    full_dbx_dir = join("dbfs:/wbeard/", dbx_dir)
    outdir = join(outdir_base, dbx_dir) if outdir is None else outdir
    dbx_fs = !$dbx fs ls $full_dbx_dir
    written = os.listdir(outdir)
    tobewritten = [f for f in dbx_fs if f not in written]
    for f in tobewritten:
        dbx_fn = join(full_dbx_dir, f)
        print('{} => {}'.format(dbx_fn, outdir))
        !$dbx fs cp $dbx_fn $outdir
    return tobewritten, written
    

def keep_moving(dbx_dir, outdir=None, outdir_base='../data/raw/', maxn=5):
    while 1:
        time.sleep(3)
        tobewritten, written = move_new(dbx_dir, outdir=outdir,
                                        outdir_base=outdir_base)

        if maxn and (len(tobewritten) + len(written) >= maxn):
            return
        print('.', end='')
        
        
vc = lambda x: Series(x).value_counts(normalize=0).sort_index()


## Load longitudinal retention

### Move

In [50]:
dbx_fs = !$dbx fs ls -l "dbfs:/wbeard/long_ret/"
dbx_fs

['file  60520254  s2_201707.msg']

In [52]:
dbx_fs = !$dbx fs ls "dbfs:/wbeard/long_ret/"
dbx_fs

['s2_201707.msg',
 's2_201708.msg',
 's2_201709.msg',
 's2_201710.msg',
 's2_201711.msg']

In [9]:
%mkdir '../data/raw/long_ret/'

In [ ]:
nfs = 5

In [53]:
%%time
for dbx_f in dbx_fs[:]:
# for dbx_f in ['s2_201707.msg', 's2_201708.msg', 's2_201711.msg']:
    dbx_loc = join("dbfs:/wbeard/long_ret/", dbx_f)
    print(dbx_loc)
    !$dbx fs cp $dbx_loc '../data/raw/long_ret/'

dbfs:/wbeard/long_ret/s2_201707.msg
dbfs:/wbeard/long_ret/s2_201708.msg
dbfs:/wbeard/long_ret/s2_201709.msg
dbfs:/wbeard/long_ret/s2_201710.msg
dbfs:/wbeard/long_ret/s2_201711.msg
CPU times: user 3.53 s, sys: 7.51 s, total: 11 s
Wall time: 3min 32s


In [67]:

    
keep_moving('long_ret')

### Convert

In [69]:
def split_ret_cts(df, col='Ret_0_1_7_30'):
    df = df.copy()
    pre, *cts = col.split('_')
    vals = df[col]
    uz = lzip(*vals)
    for ctlab, col_vals in zip(cts, zip(*vals)):
    #     print(ctlab, col_vals)
        df['{}{}'.format(pre, ctlab)] = list(col_vals)
    return df.drop(col, axis=1)

In [70]:
raw_loc = '../data/raw/long_ret/*'

In [71]:
intrm_loc = '../data/interim/long_ret/'

In [72]:
for msg in glob(raw_loc)[::]:
#     %time _df = pd.read_msgpack(msg).rename(columns={})
    %time eu.load_usage(msg, overwrite=False, transform_df=split_ret_cts, parq=True, outdir=intrm_loc, debug=0)
#     break

-> /Users/wbeard/repos/events/data/interim/long_ret/s2_201707.msg.parq
CPU times: user 3.87 s, sys: 486 ms, total: 4.36 s
Wall time: 4.4 s
-> /Users/wbeard/repos/events/data/interim/long_ret/s2_201708.msg.parq
CPU times: user 3.23 s, sys: 388 ms, total: 3.62 s
Wall time: 3.65 s
-> /Users/wbeard/repos/events/data/interim/long_ret/s2_201709.msg.parq
CPU times: user 3.59 s, sys: 432 ms, total: 4.02 s
Wall time: 4.03 s
-> /Users/wbeard/repos/events/data/interim/long_ret/s2_201710.msg.parq
CPU times: user 2.77 s, sys: 337 ms, total: 3.11 s
Wall time: 3.14 s
-> /Users/wbeard/repos/events/data/interim/long_ret/s2_201711.msg.parq
CPU times: user 1.07 s, sys: 112 ms, total: 1.18 s
Wall time: 1.19 s


## Load longitudinal seq=1

In [7]:
s1 = lambda: None
s1.raw_loc = '../data/raw/long_s1/'
s1.parq_loc = '../data/interim/long_s1/'

In [28]:
dbx_fs = !$dbx fs ls "dbfs:/wbeard/long_s1/"
print(len(dbx_fs))

1


In [29]:
%mkdir $s1.raw_loc
ls $s1.raw_loc

mkdir: ../data/raw/long_s1/: File exists


In [31]:
!rm ../data/raw/long_s1/s2_201711.msg

In [32]:
move_new('long_s1', outdir=None, outdir_base='../data/raw/')

dbfs:/wbeard/long_s1/s2_201711.msg => ../data/raw/long_s1


(['s2_201711.msg'], [])

### Convert to parq

In [33]:
def preproc(df):
    df = df.copy()
    nullbm = df.N_ev == 1
    for c in 'Seq1'.split():
        df = df.copy()
        df.loc[nullbm, c] = -1
        df[c] = df[c].astype(int)
    df.loc[nullbm, 'Ev1'] = [((-1, ''),)] * nullbm.sum()
    return df

rn2 = {c: c[:-1] for c in 'Ev1 Seq1'.split()}
keepcols = 'Day0str Last_day Locale_lst Locale_fst N_ev Ret1 Ret7 Ret30'.split()

In [39]:
lef1 = fc.FthDct('../data/interim/cid_long_s1.fth')
lef1.clear()

In [34]:
%%javascript
(new Audio('../s.mp3')).play();

<IPython.core.display.Javascript object>

In [41]:
ls -l /Users/wbeard/repos/events/data/interim/long_s1/s2_201711.msg.parq

-rw-r--r--  1 wbeard  staff  57348743 Dec  7 11:09 /Users/wbeard/repos/events/data/interim/long_s1/s2_201711.msg.parq


In [40]:
for i, msgname in enumerate(glob(join(s1.raw_loc, '*.msg'))):
    print(msgname)
    %time df = eu.load_and_flatten(msgname, cid_dct=lef1, overwrite=0, outdir=s1.parq_loc, parq=True, rn=rn2, keepcols=keepcols, preproc=preproc, verbose=not i)
    print('{:,.0f}'.format(len(df)))
lef1.save()

../data/raw/long_s1/s2_201711.msg
-> /Users/wbeard/repos/events/data/interim/long_s1/s2_201711.msg.parq
Missing:  {'Client_id', 'Seq1', 'Ev1'}
Extra:  {'Ts', 'Cid', 'Seq', 'Ev'}
CPU times: user 1min 8s, sys: 4.29 s, total: 1min 12s
Wall time: 1min 12s
4,313,915


In [25]:
%mkdir /Users/wbeard/repos/events/data/interim/long_s1

In [27]:
%%javascript
(new Audio('../s.mp3')).play();

<IPython.core.display.Javascript object>

## Load longitudinal seq=2

In [7]:
s2 = lambda: None
s2.raw_loc = '../data/raw/long_s2/'
s2.parq_loc = '../data/interim/long_s2/'

### Move raw

In [8]:
dbx_fs = !$dbx fs ls "dbfs:/wbeard/long_s2/"
print(len(dbx_fs))

5


In [9]:
!$dbx fs ls -l "dbfs:/wbeard/long_s2/"

file  168943253  s2_201707.msg
file  134326711  s2_201708.msg
file  152242853  s2_201709.msg
file  113492956  s2_201710.msg
file  115149523  s2_201711.msg


In [8]:
%mkdir $s2.raw_loc

mkdir: ../data/raw/long_s2/: File exists


In [11]:
move_new('long_s2', outdir=None, outdir_base='../data/raw/')

dbfs:/wbeard/long_s2/s2_201707.msg => ../data/raw/long_s2
dbfs:/wbeard/long_s2/s2_201708.msg => ../data/raw/long_s2
dbfs:/wbeard/long_s2/s2_201709.msg => ../data/raw/long_s2
dbfs:/wbeard/long_s2/s2_201710.msg => ../data/raw/long_s2
dbfs:/wbeard/long_s2/s2_201711.msg => ../data/raw/long_s2


(['s2_201707.msg',
  's2_201708.msg',
  's2_201709.msg',
  's2_201710.msg',
  's2_201711.msg'],
 [])

In [20]:
keep_moving('long_s2', outdir=None, outdir_base='../data/raw/')

dbfs:/wbeard/long_s2/s2_201709.msg => ../data/raw/long_s2
dbfs:/wbeard/long_s2/s2_201710.msg => ../data/raw/long_s2
dbfs:/wbeard/long_s2/s2_201711.msg => ../data/raw/long_s2


In [13]:
%%javascript
(new Audio('../s.mp3')).play();

<IPython.core.display.Javascript object>

In [9]:
%%time
# for dbx_f in dbx_fs[:]:
for dbx_f in ['s2_201707.msg', 's2_201708.msg', 's2_201711.msg']:
    dbx_loc = join("dbfs:/wbeard/long_s2/", dbx_f)
    print(dbx_loc)
    !$dbx fs cp $dbx_loc $s2.raw_loc

dbfs:/wbeard/long_s2/s2_201707.msg
dbfs:/wbeard/long_s2/s2_201708.msg
dbfs:/wbeard/long_s2/s2_201711.msg
CPU times: user 5.08 s, sys: 12.5 s, total: 17.5 s
Wall time: 5min 22s


### Convert to parq

In [17]:
# indir = '../data/raw/long_s2/'
# outdir = '../data/interim/long_s2/'

In [17]:
def preproc(df):
    df = df.copy()
    nullbm = df.N_ev == 1
    for c in 'Day_ev2 Created2 Seq2'.split():
        df = df.copy()
        df.loc[nullbm, c] = -1
        df[c] = df[c].astype(int)
    df.loc[nullbm, 'Ev2'] = [((-1, ''),)] * nullbm.sum()
    return df

rn2 = {c: c[:-1] for c in 'Ev2 Created2 Seq2'.split()}
keepcols = 'Day0str Day_ev2 Last_day Locale_lst Locale_fst N_ev Ret1 Ret7 Ret30'.split()

dfm = pd.read_msgpack(msgname)
dfm = dfm.rename(columns=bytes.decode).rename(columns=str.capitalize)

preproc(dfm[:4])

#### debug

In [33]:
df = pd.read_msgpack('../data/raw/long_s2/s2_201711.msg')[:3]  #.rename(columns=bytes.decode).rename(columns=rn2)
df.to_msgpack('/tmp/x.msg')

In [35]:
df

,b'client_id',b'Day0str',b'Day_ev2',b'Last_day',b'Locale_lst',b'Locale_fst',b'N_ev',b'Ev2',b'Created2',b'Seq2',b'Ret1'
0,05cf25b7-7f7c-45d1-b597-d28eed67cf98,20171102,20171107,20171107,en-US,en-US,2,"((0, foreground::app::), (8681, type_url::sear...",1.510070e+12,2.0,False
1,071ed95b-2f56-4de9-9789-a7158c6a9f04,20171107,20171107,20171107,en-US,en-US,2,"((101865, foreground::app::), (108873, type_qu...",1.510065e+12,2.0,False
2,129be804-e2b7-4961-b1fc-74e4aad2bab3,20171116,20171116,20171116,id-ID,id-ID,3,"((38773, foreground::app::), (148809, backgrou...",1.510827e+12,2.0,False


In [16]:
%mkdir $s2.parq_loc

In [50]:
df = eu.load_and_flatten('/tmp/x.msg', cid_dct=_le, overwrite=1, outdir='/tmp/t', parq=True, rn=rn2, keepcols=keepcols, preproc=preproc, verbose=True)

-> /tmp/t/x.msg.parq
Missing:  {'Ev2', 'Created2', 'Client_id', 'Seq2'}
Extra:  {'Ts', 'Ev', 'Created', 'Cid', 'Seq'}


In [47]:
import utils.events_utils as eu; reload(eu);
%mkdir /tmp/t
_le = defaultdict(lambda ctr=count(): next(ctr))

mkdir: /tmp/t: File exists


#### debug end

In [18]:
lef = fc.FthDct('../data/interim/cid_long_s2.fth')
lef.clear()

In [19]:
for i, msgname in enumerate(glob(join(s2.raw_loc, '*.msg'))):
    print(msgname)
    %time df = eu.load_and_flatten(msgname, cid_dct=lef, overwrite=0, outdir=s2.parq_loc, parq=True, rn=rn2, keepcols=keepcols, preproc=preproc, verbose=not i)
    print('{:,.0f}'.format(len(df)))
lef.save()

../data/raw/long_s2/s2_201707.msg
-> /Users/wbeard/repos/events/data/interim/long_s2/s2_201707.msg.parq
Missing:  {'Ev2', 'Created2', 'Client_id', 'Seq2'}
Extra:  {'Ev', 'Ts', 'Created', 'Cid', 'Seq'}
CPU times: user 1min 24s, sys: 6.18 s, total: 1min 30s
Wall time: 1min 31s
3,539,431
../data/raw/long_s2/s2_201708.msg
-> /Users/wbeard/repos/events/data/interim/long_s2/s2_201708.msg.parq
CPU times: user 1min 4s, sys: 3.86 s, total: 1min 8s
Wall time: 1min 9s
2,695,238
../data/raw/long_s2/s2_201709.msg
-> /Users/wbeard/repos/events/data/interim/long_s2/s2_201709.msg.parq
CPU times: user 1min 18s, sys: 5.21 s, total: 1min 23s
Wall time: 1min 25s
3,075,929
../data/raw/long_s2/s2_201710.msg
-> /Users/wbeard/repos/events/data/interim/long_s2/s2_201710.msg.parq
CPU times: user 1min 11s, sys: 5.25 s, total: 1min 16s
Wall time: 1min 18s
2,325,852
../data/raw/long_s2/s2_201711.msg
-> /Users/wbeard/repos/events/data/interim/long_s2/s2_201711.msg.parq
CPU times: user 1min 10s, sys: 5.23 s, total: 

In [104]:
df[:3]

,Day1,Day_ev2,Last_day,N_ev,Cid,Seq,Created,Ev,Ts
0,20171104,-1,20171104,1,2768527,-1,-1,,-1
1,20171101,20171101,20171104,10,2768528,9,1509781325304,intent_url::app::,0
2,20171101,20171101,20171104,10,2768528,9,1509781325304,foreground::app::,710


## Load Onboarding A/B

%mkdir '../data/raw/onboarding'

In [13]:
move_new('onboarding', outdir=None, outdir_base='../data/raw/')

dbfs:/wbeard/onboarding/maxday_20180201.msg => ../data/raw/onboarding


(['maxday_20180201.msg'], ['maxday_20171230.msg'])

In [14]:
df = pd.read_msgpack('../data/raw/onboarding/maxday_20180201.msg')

In [16]:
/len df

35955

In [17]:
df[:4]

,b'Day0str',b'Day0',b'Onboarding',b'Ret1',b'Ret7',b'Se0',b'Se_fin',b'Locale_fst'
0,20171227,2017-12-27,"(ios_onboarding_v1, 0)",True,True,DuckDuckGo,DuckDuckGo,es-ES
1,20180101,2018-01-01,"(ios_onboarding_v1, 0)",False,False,Amazon.com,Google,en-US
2,20171220,2017-12-20,"(ios_onboarding_v1, 0)",False,False,Amazon.com,Amazon.com,en-US
3,20180104,2018-01-04,"(previous_first_run, 0)",False,False,Amazon.com,Yahoo,en-US


In [8]:
onboarding/maxday_20171230.msg

NameError: name 'onboarding' is not defined

## OOM

In [1]:
%mkdir '../data/raw/new_profile'

In [ ]:
def confirm_fns(fs):
    # - sort by date
    print()

In [19]:
move_new('oom/pings', outdir='../data/raw/oom/pings/', outdir_base='../data/raw/oom/pings/')

dbfs:/wbeard/oom/pings/p_0125.json => ../data/raw/oom/pings/


(['p_0125.json'],
 ['p_0115.json',
  'p_0116.json',
  'p_0117.json',
  'p_0118.json',
  'p_0119.json',
  'p_0120.json',
  'p_0121.json',
  'p_0122.json',
  'p_0124_slim.json'])

In [ ]:
tmpdecomp2
dbx fs cp dbfs:/wbeard/oom/main_20180101_20180201.msg  ../../ts-focus/data/raw/oom/

In [11]:
mv_s3_pq(from_dir='new_profile', to_dir='data/new_profile', fn='nprof_20170101_20180101.parq')

Found pq file part-00000-tid-2707467087415157492-a60a39e7-af1c-47bf-b87e-a7e131f66374-254238-c000.snappy.parquet
download: s3://mozilla-databricks-telemetry-test/wbeard/new_profile/nprof_20170101_20180101.parq/part-00000-tid-2707467087415157492-a60a39e7-af1c-47bf-b87e-a7e131f66374-254238-c000.snappy.parquet to data/new_profile/nprof_20170101_20180101.parq


'data/new_profile/nprof_20170101_20180101.parq'

In [9]:
%mkdir data/p26_8

In [12]:
!aws s3 sync 's3://mozilla-databricks-telemetry-test/wbeard/crash_pings/p_201801' data/p29_0202

download: s3://mozilla-databricks-telemetry-test/wbeard/crash_pings/p_20180126_20180128_eq_42.parq/_SUCCESS to data/p26_8/_SUCCESS
download: s3://mozilla-databricks-telemetry-test/wbeard/crash_pings/p_20180126_20180128_eq_42.parq/_committed_6840194297520358502 to data/p26_8/_committed_6840194297520358502
download: s3://mozilla-databricks-telemetry-test/wbeard/crash_pings/p_20180126_20180128_eq_42.parq/_started_6840194297520358502 to data/p26_8/_started_6840194297520358502
download: s3://mozilla-databricks-telemetry-test/wbeard/crash_pings/p_20180126_20180128_eq_42.parq/part-00002-tid-6840194297520358502-d54968ad-db37-4285-8f8c-e2e58808f762-0-c000.snappy.parquet to data/p26_8/part-00002-tid-6840194297520358502-d54968ad-db37-4285-8f8c-e2e58808f762-0-c000.snappy.parquet
download: s3://mozilla-databricks-telemetry-test/wbeard/crash_pings/p_20180126_20180128_eq_42.parq/part-00000-tid-6840194297520358502-d54968ad-db37-4285-8f8c-e2e58808f762-0-c000.snappy.parquet to data/p26_8/part-00000-tid-

## Custom tab Focus

In [12]:
%mkdir ../data/cust_tab

In [13]:
s3share = 's3://mozilla-databricks-telemetry-test/wbeard/'

In [17]:
pwd

'/Users/wbeard/repos/ts-focus/notebooks'

In [22]:
mv data/share/d1ret.parq ../data/share/

In [18]:
ls data/share/

d1ret.parq


In [18]:
mv_s3_pq(from_dir='focus_custom_tab', to_dir='../data/cust_tab', fn='day_meth_20180315.parq')

Found pq file part-00000-tid-794809823562251692-12a303db-ac09-44f3-979a-06ce9fe57e61-2192613-c000.snappy.parquet
download: s3://mozilla-databricks-telemetry-test/wbeard/focus_custom_tab/day_meth_20180315.parq/part-00000-tid-794809823562251692-12a303db-ac09-44f3-979a-06ce9fe57e61-2192613-c000.snappy.parquet to ../data/cust_tab/day_meth_20180315.parq


'../data/cust_tab/day_meth_20180315.parq'

## Share A/B test

In [7]:
%mkdir ../data/share

In [9]:
s3share = 's3://mozilla-databricks-telemetry-test/wbeard/focus_share'

In [17]:
pwd

'/Users/wbeard/repos/ts-focus/notebooks'

In [19]:
ls 

In [22]:
mv data/share/d1ret.parq ../data/share/

In [18]:
ls data/share/

d1ret.parq


In [24]:
mv_s3_pq(from_dir='focus_share', to_dir='../data/share', fn='d1ret2.parq')

Found pq file part-00000-tid-3778345720412391848-b9b23223-a85e-4439-8ec3-9386fcf823af-288996-c000.snappy.parquet
download: s3://mozilla-databricks-telemetry-test/wbeard/focus_share/d1ret2.parq/part-00000-tid-3778345720412391848-b9b23223-a85e-4439-8ec3-9386fcf823af-288996-c000.snappy.parquet to ../data/share/d1ret2.parq


'../data/share/d1ret2.parq'

## Tre

In [63]:
toks = re.split('[^A-Za-z]', 'mozilla-databricks-telemetry-test/wbeard/crash_pings/p_20180126_20180128_eq_42.parq')
toks = lfilter(None, toks)

def gen_str(upto=1000):
    zero = nr.rand() > .7
    if zero:
        return []
    n = nr.randint(0, upto)
    return nr.choice(toks, size=n)

nr.seed(0)
str_l = [gen_str() for _ in range(100000)]

sl = Series(str_l)
sa = sl.map(lambda x: np.array(x, dtype=object))

_dct = defaultdict(lambda ctr=count(): next(ctr))
si = sa.map(lambda xs: np.array(lmap(_dct.__getitem__, xs), dtype=int))
Series.d = lambda x: x.to_frame().rename(columns=str)

In [64]:
fn = "/tmp/x.parq"
!rm $fn
sad = sa.d()
%time sad.to_parquet(fn)
sid = si.d()
%time sid.to_parquet(fn)

CPU times: user 5.57 s, sys: 1.63 s, total: 7.2 s
Wall time: 7.75 s
CPU times: user 2.31 s, sys: 1.12 s, total: 3.42 s
Wall time: 3.43 s


## Retention

In [7]:
base = 's3://net-mozaws-prod-us-west-2-pipeline-analysis/sguha/tmp/'
pqdir = 'retention_samples_2016-08-01_2017-12-01.pqt'
mv_s3_pq(from_dir='', to_dir='../../tte/data/raw/', fn=pqdir, s3base=base)

^C
ERROR: File s3://net-mozaws-prod-us-west-2-pipeline-analysis/sguha/tmp/retention_samples_2016-08-01_2017-12-01.pqt not found

Possible options:
	                            PRE a11pq/
	                           PRE a11pqall/
	                           PRE bsexample/
	                           PRE bsexample2/
	                           PRE churnsamples/
	                           PRE one/
	                           PRE pqs/
	                           PRE pqs2/
	                           PRE pqs2obs/
	                           PRE retention_samples_2016-08-01_2017-12-01.csv/
	                           PRE retention_samples_2016-08-01_2017-12-01.pqt/
	                           PRE x.csv/
	                           PRE x2.csv/


## Ipc

In [6]:
ls ../../ipc/data/raw/

In [8]:
local_dir = '../../ipc/data/raw/'
base = 's3://net-mozaws-prod-us-west-2-pipeline-analysis/wbeard/ipc/'

In [32]:
# base = 'dbfs:/wbeard/apb/'
pqdir = 'samp.json'
# pqdir = 'abc'

mv_s3_pq(from_dir='', to_dir=local_dir, fn=pqdir, s3base=base, ext='json')

Found pq file part-00000-tid-3179433506591924477-7c90a292-abf3-44e3-98b3-7213773c5a1d-10458612-c000.json
download: s3://net-mozaws-prod-us-west-2-pipeline-analysis/wbeard/ipc/samp.json/part-00000-tid-3179433506591924477-7c90a292-abf3-44e3-98b3-7213773c5a1d-10458612-c000.json to ../data/raw/samp.json


'../../ipc/data/raw/samp.json'

In [24]:
json_file = join(base, pqdir)
print(json_file)
!aws s3 cp "$json_file" "$local_dir"

s3://net-mozaws-prod-us-west-2-pipeline-analysis/wbeard/ipc/samp.json
fatal error: An error occurred (404) when calling the HeadObject operation: Key "wbeard/ipc/samp.json" does not exist


In [27]:
!aws s3 ls "$json_file/"

2018-11-28 12:23:24          0 _SUCCESS
2018-11-28 12:23:24        117 _committed_3179433506591924477
2018-11-28 12:23:23          0 _started_3179433506591924477
2018-11-28 12:23:24      51818 part-00000-tid-3179433506591924477-7c90a292-abf3-44e3-98b3-7213773c5a1d-10458612-c000.json
